In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,30922
2,Huelva,Confirmados PDIA 14 días,1376
3,Huelva,Tasa PDIA 14 días,"268,13726445427443"
4,Huelva,Confirmados PDIA 7 días,682
5,Huelva,Total Confirmados,31102
6,Huelva,Curados,28509
7,Huelva,Fallecidos,378


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  30922.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  8159.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 575 personas en los últimos 7 días 

Un positivo PCR cada 331 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,30922.0,682.0,1376.0,513170.0,132.899429,268.137264,269.0
Huelva-Costa,17814.0,365.0,669.0,289548.0,126.058546,231.049774,142.0
Condado-Campiña,10042.0,266.0,599.0,156231.0,170.260704,383.406622,106.0
Huelva (capital),8159.0,250.0,434.0,143837.0,173.807852,301.730431,101.0
Almonte,1788.0,58.0,134.0,24507.0,236.667075,546.782552,29.0
Sierra de Huelva-Andévalo Central,2720.0,49.0,103.0,67391.0,72.710006,152.839400,20.0
Moguer,1268.0,50.0,112.0,21867.0,228.655051,512.187314,20.0
Aljaraque,1074.0,35.0,53.0,21474.0,162.987799,246.810096,17.0
Niebla,231.0,20.0,26.0,4116.0,485.908649,631.681244,14.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villarrasa,167.0,4.0,17.0,2211.0,180.913614,768.882858,1.0
Cabezas Rubias,37.0,1.0,5.0,706.0,141.643059,708.215297,0.0
Palma del Condado (La),1141.0,26.0,75.0,10801.0,240.718452,694.380150,3.0
Niebla,231.0,20.0,26.0,4116.0,485.908649,631.681244,14.0
Almonte,1788.0,58.0,134.0,24507.0,236.667075,546.782552,29.0
Moguer,1268.0,50.0,112.0,21867.0,228.655051,512.187314,20.0
Bollullos Par del Condado,970.0,25.0,73.0,14387.0,173.767985,507.402516,11.0
Calañas,189.0,4.0,12.0,2768.0,144.508671,433.526012,0.0
Manzanilla,98.0,3.0,9.0,2118.0,141.643059,424.929178,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cabezas Rubias,37.0,1.0,5.0,706.0,141.643059,708.215297,0.0,0.200000
Villarrasa,167.0,4.0,17.0,2211.0,180.913614,768.882858,1.0,0.235294
Trigueros,297.0,5.0,19.0,7862.0,63.597049,241.668787,3.0,0.263158
San Bartolomé de la Torre,294.0,3.0,11.0,3761.0,79.766020,292.475405,1.0,0.272727
Manzanilla,98.0,3.0,9.0,2118.0,141.643059,424.929178,0.0,0.333333
Villalba del Alcor,481.0,1.0,3.0,3366.0,29.708853,89.126560,0.0,0.333333
Calañas,189.0,4.0,12.0,2768.0,144.508671,433.526012,0.0,0.333333
Bollullos Par del Condado,970.0,25.0,73.0,14387.0,173.767985,507.402516,11.0,0.342466
Palma del Condado (La),1141.0,26.0,75.0,10801.0,240.718452,694.380150,3.0,0.346667
